# Load Packages

In [1]:
#Using smth means: pre-compile and load smth as a package.
using Pkg;
#Set the Gurobi path
ENV["GUROBI_HOME"] = "/Library/gurobi912/macos64/"
#For first timers (comment after): compile Gurobi with the given path
#Pkg.build("Gurobi")
#Add the packages that may not be installed.
Pkg.add(["Latexify", "Polyhedra", "Plots", "Makie", "CDDLib", "JuMP", "Gurobi", "AbstractPlotting", "JLD", "HDF5"])
#Load the packages
using JuMP, Latexify, Gurobi, LinearAlgebra, Polyhedra, CDDLib, Plots, SparseArrays, Makie, AbstractPlotting, JLD, HDF5

display("Initialization done! All set :-)")

    Updating registry at `C:\Users\Alexr\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
  No Changes to `C:\Users\Alexr\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Alexr\.julia\environments\v1.6\Manifest.toml`


"Initialization done! All set :-)"

┌ Warning: Everything from AbstractPlotting.jl got moved to Makie.jl. 
│ Have a look at the README for information on how to upgrade: 
│ https://github.com/JuliaPlots/AbstractPlotting.jl
└ @ AbstractPlotting C:\Users\Alexr\.julia\packages\AbstractPlotting\2A5iv\src\AbstractPlotting.jl:261


# Load Data

In [2]:
data_file_1 = load("../data/X1-3.jld")["X"] 
data_file_2 = load("../data/X2-2.jld")["X"]
data_file_3 = load("../data/X3-2.jld")["X"]
data = data_file_1

25×2 Matrix{Float64}:
 -9.31431     21.8525
 -5.66901     22.8257
 -5.58994     21.7019
 -6.48529     20.4481
 -4.64804    -13.5705
 -6.92318    -13.4546
 -4.92195    -13.169
 -6.18895    -15.1609
 -6.38359    -14.1112
 -4.83052    -13.3698
 -5.54885    -13.7464
 -6.03902    -13.1761
 -9.36696    -13.6319
 -1.01326     -5.99053
  0.0875803   -4.8667
  1.34522     -5.53625
  1.02764     -6.47446
  0.180654    -4.69903
  0.968761    -4.01118
  2.14521     -6.54907
 -3.81989      1.50499
 -4.94545      1.28581
 -3.29144      2.76385
 -2.36855      2.11314
 -4.05179      1.67859

# Create Gurobi Problem

In [3]:
problem1 = Model(Gurobi.Optimizer)

Academic license - for non-commercial use only


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

# Calculate Distance Matrix

In [4]:
k = Array{Float64}(undef, size(data,1), size(data,1))
for i = 1:size(data,1)
    for j = 1:size(data,1)
        k[i,j] = sqrt((data[i,1]-data[j,1])^2 + (data[i,2]-data[j,2])^2)
    end
end

# Create Constraints

In [5]:
max_clients = Int(size(data,1))
n_max = 8
n_min = 2
d_max = 10
max_depots = Int(ceil(max_clients/n_min))

13

# Create y

In [6]:
@variable(problem1, y[1:max_depots], base_name = "y", binary=true)

13-element Vector{VariableRef}:
 y[1]
 y[2]
 y[3]
 y[4]
 y[5]
 y[6]
 y[7]
 y[8]
 y[9]
 y[10]
 y[11]
 y[12]
 y[13]

# Create x

In [7]:
@variable(problem1, x[1:max_clients,1:max_depots], base_name = "x", binary=true)

25×13 Matrix{VariableRef}:
 x[1,1]   x[1,2]   x[1,3]   x[1,4]   x[1,5]   …  x[1,11]   x[1,12]   x[1,13]
 x[2,1]   x[2,2]   x[2,3]   x[2,4]   x[2,5]      x[2,11]   x[2,12]   x[2,13]
 x[3,1]   x[3,2]   x[3,3]   x[3,4]   x[3,5]      x[3,11]   x[3,12]   x[3,13]
 x[4,1]   x[4,2]   x[4,3]   x[4,4]   x[4,5]      x[4,11]   x[4,12]   x[4,13]
 x[5,1]   x[5,2]   x[5,3]   x[5,4]   x[5,5]      x[5,11]   x[5,12]   x[5,13]
 x[6,1]   x[6,2]   x[6,3]   x[6,4]   x[6,5]   …  x[6,11]   x[6,12]   x[6,13]
 x[7,1]   x[7,2]   x[7,3]   x[7,4]   x[7,5]      x[7,11]   x[7,12]   x[7,13]
 x[8,1]   x[8,2]   x[8,3]   x[8,4]   x[8,5]      x[8,11]   x[8,12]   x[8,13]
 x[9,1]   x[9,2]   x[9,3]   x[9,4]   x[9,5]      x[9,11]   x[9,12]   x[9,13]
 x[10,1]  x[10,2]  x[10,3]  x[10,4]  x[10,5]     x[10,11]  x[10,12]  x[10,13]
 x[11,1]  x[11,2]  x[11,3]  x[11,4]  x[11,5]  …  x[11,11]  x[11,12]  x[11,13]
 x[12,1]  x[12,2]  x[12,3]  x[12,4]  x[12,5]     x[12,11]  x[12,12]  x[12,13]
 x[13,1]  x[13,2]  x[13,3]  x[13,4]  x[13,5]  

# Add objective

In [8]:
@objective(problem1, Min, sum(y))

y[1] + y[2] + y[3] + y[4] + y[5] + y[6] + y[7] + y[8] + y[9] + y[10] + y[11] + y[12] + y[13]

# Add Constraint 1
$$\sum_{c=1}^nx_{cd}\leq n_{max}*y_d\forall d$$

In [9]:
for d in 1:max_depots
    constraint_expr = @expression(problem1, 0)
    for c in 1:max_clients
        constraint_expr += x[c,d]
    end
    @constraint(problem1, constraint_expr<=n_max*y[d])
end

# Add constraint 2
$$\sum_{c=1}^nx_{cd}\geq n_{min}*y_d\forall d $$

In [10]:
for d in 1:max_depots
    constraint_expr = @expression(problem1, 0)
    for c in 1:max_clients
        constraint_expr += x[c,d]
    end
    @constraint(problem1, constraint_expr>=n_min*y[d])
end

# Constraint 3
$$\sum_{d=1}^nx_{cd}= 1\forall c$$

In [11]:
for c in 1:max_clients
    constraint_expr = @expression(problem1, 0)
    for d in 1:max_depots
        constraint_expr += x[c,d]
    end
    @constraint(problem1, constraint_expr==1)
end

# Constraint 4
$$\sum_{c_1=1}^nk_{{c_1}{c_2}}*x_{c_1d}*x_{c_2d}\leq d_{max}\forall c_2 \in C, \forall d \in D$$

In [13]:
for d in 1:max_depots
    for c1 in 1:max_clients
        constraint_expr = @expression(problem1, 0)
        for c2 in 1:c1-1
            constraint_expr += k[c1,c2]*x[c1,d]*x[c2,d]
        end
        @constraint(problem1, constraint_expr<=d_max)
    end
end

# Solve

In [17]:
optimize!(problem1)

Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 64 rows, 338 columns and 1001 nonzeros
Model fingerprint: 0xeee41648
Model has 312 quadratic constraints
Variable types: 0 continuous, 338 integer (338 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  QMatrix range    [2e-01, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+01]
  QRHS range       [1e+01, 1e+01]
Presolve removed 13 rows and 0 columns
Presolve time: 0.06s
Presolved: 51 rows, 338 columns, 1001 nonzeros
Presolved model has 273 quadratic constraint(s)
Variable types: 0 continuous, 338 integer (338 binary)

Root relaxation: objective 3.125000e+00, 78 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    3.12500    0    1          -    3.12500 